# Chapter 5
# Numerical Integration and Differentiation

In many computational economic applications, one must compute the definite integral
of a real-valued function f with respect to a "weighting" function w over an interval
$I$ of $R^n$:

$$\int_I f(x)w(x) dx$$


The weighting function may be the identity, $w = 1$, in which case the integral represents
the area under the function f. In other applications, w may be the probability
density of a random variable $\tilde X$
, in which case the integral represents the expectation
of $f( \tilde X)$ when $I$ repesents the whole support of $\tilde X$.





In this chapter, we discuss three classes of numerical integration or numerical
quadrature methods<sup>1</sup>. All methods approximate the integral with a weighted sum of
function values:

$$\int_I f(x) w(x)dx \approx \sum_{i=0}^{n} w_i f(x_i)\thinspace .$$

<sup>1</sup>Quadrature is a historical mathematical term that means calculating area. 

The methods differ only in how the *quadrature weights* $wi$ and the *quadrature nodes*
$xi$ are chosen.

**Newton-Cotes** methods approximate the integrand f between nodes
using low order polynomials, and sum the integrals of the polynomials to estimate
the integral of f. Newton-Cotes methods are easy to implement, but are not particularly
eÆcient for computing the integral of a smooth function.

**Gaussian quadrature**
methods choose the nodes and weights to satisfy moment matching conditions, and
are more powerful than Newton-Cotes methods if the integrand is smooth.

**Monte Carlo and quasi-Monte Carlo integration** methods use "random" or "equidistributed"
nodes, and are simple to implement and are especially useful if the integration domain
is of high dimension or irregularly shaped.


In this chapter, we also present an overview of how to compute *finite difference*
approximations for the derivatives of a real-valued function. 

As we have seen in
previous chapters, it is often desirable to compute derivatives numerically because
analytic derivative expressions are difficult or impossible to derive, or expensive to
evaluate. 

Finite difference methods can also be used to solve differential equations,
which arise frequently in dynamic economic models, especially models formulated in
continuous time. In this chapter, we introduce numerical methods for differential
equations and illustrate their application to *initial value problems*.

In [2]:
# https://github.com/QuantEcon/QuantEcon.py/blob/488b7b3b9117cfd9bfc71c187efc87c39fc5b459/quantecon/quad.py
"""
Filename: quad.py
Authors: Chase Coleman, Spencer Lyon
Date: 2014-07-01
Defining various quadrature routines.
Based on the quadrature routines found in the CompEcon toolbox by
Miranda and Fackler.
References
----------
Miranda, Mario J, and Paul L Fackler. Applied Computational Economics
and Finance, MIT Press, 2002.
"""

from __future__ import division

import math
import numpy as np
import scipy.linalg as la
from scipy.special import gammaln
import sympy as sym
#from .ce_util import ckron, gridmake


from functools import reduce

In [3]:
def ckron(*arrays):
    """
    Repeatedly applies the np.kron function to an arbitrary number of
    input arrays
    Parameters
    ----------
    *arrays : tuple/list of np.ndarray
    Returns
    -------
    out : np.ndarray
        The result of repeated kronecker products
    Notes
    -----
    Based of original function `ckron` in CompEcon toolbox by Miranda
    and Fackler
    References
    ----------
    Miranda, Mario J, and Paul L Fackler. Applied Computational
    Economics and Finance, MIT Press, 2002.
    """
    return reduce(np.kron, arrays)

def gridmake(*arrays):
    """
    TODO: finish this docstring
    Notes
    -----
    Based of original function ``gridmake`` in CompEcon toolbox by
    Miranda and Fackler
    References
    ----------
    Miranda, Mario J, and Paul L Fackler. Applied Computational Economics
    and Finance, MIT Press, 2002.
    """
    if all([i.ndim == 1 for i in arrays]):
        d = len(arrays)
        if d == 2:
            out = _gridmake2(*arrays)
        else:
            out = _gridmake2(arrays[0], arrays[1])
            for arr in arrays[2:]:
                out = _gridmake2(out, arr)

        return out
    else:
        raise NotImplementedError("Come back here")

        
def _gridmake2(x1, x2):
    """
    TODO: finish this docstring
    Notes
    -----
    Based of original function ``gridmake2`` in CompEcon toolbox by
    Miranda and Fackler
    References
    ----------
    Miranda, Mario J, and Paul L Fackler. Applied Computational Economics
    and Finance, MIT Press, 2002.
    """
    if x1.ndim == 1 and x2.ndim == 1:
        return np.column_stack([np.tile(x1, x2.shape[0]),
                               np.repeat(x2, x1.shape[0])])
    elif x1.ndim > 1 and x2.ndim == 1:
        first = np.tile(x1, (x2.shape[0], 1))
        second = np.repeat(x2, x1.shape[0])
        return np.column_stack([first, second])
    else:
        raise NotImplementedError("Come back here")        

def _qnwtrap1(n, a, b):
    """
    Compute univariate trapezoid rule quadrature nodes and weights
    Parameters
    ----------
    n : int
        The number of nodes
    a : int
        The lower endpoint
    b : int
        The upper endpoint
    Returns
    -------
    nodes : np.ndarray(dtype=float)
        An n element array of nodes
    nodes : np.ndarray(dtype=float)
        An n element array of weights
    Notes
    -----
    Based of original function ``qnwtrap1`` in CompEcon toolbox by
    Miranda and Fackler
    References
    ----------
    Miranda, Mario J, and Paul L Fackler. Applied Computational
    Economics and Finance, MIT Press, 2002.
    """
    if n < 1:
        raise ValueError("n must be at least one")

    nodes = np.linspace(a, b, n)
    dx = nodes[1] - nodes[0]

    weights = dx * np.ones(n)
    weights[0] *= 0.5
    weights[-1] *= 0.5

    return nodes, weights
        

def _qnwsimp1(n, a, b):
    """
    Compute univariate Simpson quadrature nodes and weights
    Parameters
    ----------
    n : int
        The number of nodes
    a : int
        The lower endpoint
    b : int
        The upper endpoint
    Returns
    -------
    nodes : np.ndarray(dtype=float)
        An n element array of nodes
    nodes : np.ndarray(dtype=float)
        An n element array of weights
    Notes
    -----
    Based of original function ``qnwsimp1`` in CompEcon toolbox by
    Miranda and Fackler
    References
    ----------
    Miranda, Mario J, and Paul L Fackler. Applied Computational
    Economics and Finance, MIT Press, 2002.
    """
    if n % 2 == 0:
        print("WARNING qnwsimp: n must be an odd integer. Increasing by 1")
        n += 1

    nodes = np.linspace(a, b, n)
    dx = nodes[1] - nodes[0]
    weights = np.tile([2.0, 4.0], (n + 1) // 2)
    weights = weights[:n]
    weights[0] = weights[-1] = 1
    weights = (dx / 3.0) * weights

    return nodes, weights    
    
        
def _qnwlege1(n, a, b):
    """
    Compute univariate Guass-Legendre quadrature nodes and weights
    Parameters
    ----------
    n : int
        The number of nodes
    a : int
        The lower endpoint
    b : int
        The upper endpoint
    Returns
    -------
    nodes : np.ndarray(dtype=float)
        An n element array of nodes
    nodes : np.ndarray(dtype=float)
        An n element array of weights
    Notes
    -----
    Based of original function ``qnwlege1`` in CompEcon toolbox by
    Miranda and Fackler
    References
    ----------
    Miranda, Mario J, and Paul L Fackler. Applied Computational
    Economics and Finance, MIT Press, 2002.
    """
    # import ipdb; ipdb.set_trace()
    maxit = 100
    m = np.fix((n + 1) / 2.0).astype(int)
    xm = 0.5 * (b + a)
    xl = 0.5 * (b - a)
    nodes = np.zeros(n)

    weights = nodes.copy()
    i = np.arange(m, dtype='int')

    z = np.cos(np.pi * ((i + 1.0) - 0.25) / (n + 0.5))

    for its in range(maxit):
        p1 = 1.0
        p2 = 0.0
        for j in range(1, n+1):
            p3 = p2
            p2 = p1
            p1 = ((2 * j - 1) * z * p2 - (j - 1) * p3) / j

        pp = n * (z * p1 - p2)/(z * z - 1.0)
        z1 = z.copy()
        z = z1 - p1/pp
        if all(np.abs(z - z1) < 1e-14):
            break

    if its == maxit - 1:
        raise ValueError("Maximum iterations in _qnwlege1")

    nodes[i] = xm - xl * z
    nodes[- i - 1] = xm + xl * z

    weights[i] = 2 * xl / ((1 - z * z) * pp * pp)
    weights[- i - 1] = weights[i]

    return nodes, weights





        
def _make_multidim_func(one_d_func, n, *args):
    """
    A helper function to cut down on code repetition. Almost all of the
    code in qnwcheb, qnwlege, qnwsimp, qnwtrap is just dealing
    various forms of input arguments and then shelling out to the
    corresponding 1d version of the function.
    This routine does all the argument checking and passes things
    through the appropriate 1d function before using a tensor product
    to combine weights and nodes.
    Parameters
    ----------
    one_d_func : function
        The 1d function to be called along each dimension
    n : int or array_like(float)
        A length-d iterable of the number of nodes in each dimension
    args :
        These are the arguments to various qnw____ functions.  For the
        majority of the functions this is just a and b, but some differ.
    Returns
    -------
    func : function
        The multi-dimensional version of the parameter ``one_d_func``
    """
    args = list(args)
    n = np.asarray(n)
    args = list(map(np.asarray, args))

    if all([x.size == 1 for x in [n] + args]):
        return one_d_func(n, *args)

    d = n.size

    for i in range(len(args)):
        if args[i].size == 1:
            args[i] = np.repeat(args[i], d)

    nodes = []
    weights = []

    for i in range(d):
        ai = [x[i] for x in args]
        _1d = one_d_func(n[i], *ai)
        nodes.append(_1d[0])
        weights.append(_1d[1])

    weights = ckron(*weights[::-1])  # reverse ordered tensor product

    nodes = gridmake(*nodes)
    return nodes, weights    

## 5.1 Newton-Cotes Methods




reference: 


https://nbviewer.jupyter.org/github/birocoles/Disciplina-metodos-computacionais/blob/master/Content/newton-cotes.ipynb

Newton-Cotes quadrature methods are designed to approximate the integral of a realvalued
function $f$ defined on a bounded interval $[a; b]$ of the real line. Newton-Cotes
methods approximate the integrand $f$ between nodes using *low order polynomials*,
and sum the integrals of the polynomials to form an estimate the integral of f. 

Two
Newton-Cotes rules are widely used in practice: the **trapezoid rule and Simpson's
rule**. Both rules are very easy to implement and are typically adequate for computing
the area under a continuous function.

### trapezoid rule

![](http://mathworld.wolfram.com/images/eps-gif/TrapezoidalRule_1000.gif)

http://mathworld.wolfram.com/Newton-CotesFormulas.html

https://en.wikipedia.org/wiki/Newton%E2%80%93Cotes_formulas

The trapezoid rule partitions the interval [a; b] into subintervals of equal length, approximates
f over each subinterval using linear interpolants, and then sums the areas under the
linear segments. The trapezoid rule draws its name from the fact that the area under f is
approximated by a series of trapezoids.

$$\int \limits_{x_i}^{x_{i+1}} f \, (x) \, d x \approx \frac{h}{2} \left[ f(x_i) + f(x_{i+1})  \right]\: .$$

where $x_i = a + (i-1)h$, with $h$ (called the step size) equal to   $ h=(b − a) / (n-1)$. The $w_i$ are called weights.

$$\int _{a}^{b}f(x)\,dx\approx \sum _{{i=1}}^{{n-1}}w_{i}\,f(x_{i}).$$


where $w_1 = w_n = h/2$ and $w_i = h$, otherwise.


In [4]:
%%latex
\begin{align}
\int_a^b f(x)\,dx &= \int_{x_0}^{x_1} f(x) dx + \int_{x_1}^{x_2} f(x) dx + \ldots + \int_{x_{n-1}}^{x_n} f(x) dx,     \nonumber \\ 
                  &\approx h \frac{f(x_0) + f(x_1)}{2} +
		  h \frac{f(x_1) + f(x_2)}{2} + \ldots + \nonumber \\ 
		  &\quad h \frac{f(x_{n-1}) + f(x_n)}{2} 
\end{align}

<IPython.core.display.Latex object>



$$\int_a^b f(x)\,dx \approx  
\frac{h}{2}\left[f(x_0) + 2 f(x_1) + 2 f(x_2) + \ldots + 2 f(x_{n-1}) + f(x_n)\right]              
$$




$$
\int_a^b f(x)\,dx \approx h \left[\frac{1}{2}f(x_0) + \sum_{i=1}^{n-1}f(x_i) + \frac{1}{2}f(x_n) \right] \thinspace .
$$

For example, when $n = 2$


$${\frac  {b-a}{2}}(f_{0}+f_{1})$$

In [ ]:
def trapezoidal(f, a, b, n):
    h = float(b-a)/n
    result = 0.5*f(a) + 0.5*f(b)
    for i in range(1, n):
        result += f(a + i*h)
    result *= h
    return result

In [ ]:
def _qnwtrap1(n, a, b):
    """
    Compute univariate trapezoid rule quadrature nodes and weights
    Parameters
    ----------
    n : int
        The number of nodes
    a : int
        The lower endpoint
    b : int
        The upper endpoint
    Returns
    -------
    nodes : np.ndarray(dtype=float)
        An n element array of nodes
    nodes : np.ndarray(dtype=float)
        An n element array of weights
    Notes
    -----
    Based of original function ``qnwtrap1`` in CompEcon toolbox by
    Miranda and Fackler
    References
    ----------
    Miranda, Mario J, and Paul L Fackler. Applied Computational
    Economics and Finance, MIT Press, 2002.
    """
    if n < 1:
        raise ValueError("n must be at least one")

    nodes = np.linspace(a, b, n)
    dx = nodes[1] - nodes[0]

    weights = dx * np.ones(n)
    weights[0] *= 0.5
    weights[-1] *= 0.5

    return nodes, weights

The trapezoid rule is simple and robust. It is said to be first order exact because, if
not for rounding error, it will exactly compute the integral of any first order polynomial,
that is, a line. In general, if the integrand f is smooth, the trapezoid rule will yield an
approximation error that is $O(h^2)$, that is, the error shrinks quadratically with the width of
the subintervals.

In [4]:
def qnwtrap(n, a, b):
    """
    Computes multivariate trapezoid rule quadrature nodes and weights.
    Parameters
    ----------
    n : int or array_like(float)
        A length-d iterable of the number of nodes in each dimension
    a : scalar or array_like(float)
        A length-d iterable of lower endpoints. If a scalar is given,
        that constant is repeated d times, where d is the number of
        dimensions
    b : scalar or array_like(float)
        A length-d iterable of upper endpoints. If a scalar is given,
        that constant is repeated d times, where d is the number of
        dimensions
    Returns
    -------
    nodes : np.ndarray(dtype=float)
        Quadrature nodes
    weights : np.ndarray(dtype=float)
        Weights for quadrature nodes
    Notes
    -----
    Based of original function ``qnwtrap`` in CompEcon toolbox by
    Miranda and Fackler
    References
    ----------
    Miranda, Mario J, and Paul L Fackler. Applied Computational
    Economics and Finance, MIT Press, 2002.
    """
    return _make_multidim_func(_qnwtrap1, n, a, b)

### Simpson's rule

Simpson's rule is based on piece-wise quadratic, rather than piece-wise linear, approximations
to the integrand $f$.



![](https://upload.wikimedia.org/wikipedia/commons/thumb/c/ca/Simpsons_method_illustration.svg/440px-Simpsons_method_illustration.svg.png)

`Simpson's rule can be derived by approximating the integrand f (x) (in blue) by the quadratic interpolant P(x) (in red).`


https://en.wikipedia.org/wiki/Simpson%27s_rule




The area under this quadratic function provides an estimate of the area under f over the
subinterval:

$$\int \limits_{x_{2j-1}}^{x_{2j+1}} f(x)\, d x \approx \frac{h}{3} \left[ \, f(x_{2j-1}) + 4 \, f(x_{2j}) + f(x_{2j+1}) \, \right]$$

where $x_i = a + (i-1)h$, with $h$ (called the step size) equal to   $ h=(b − a) / (n-1)$ and n is odd. The $w_i$ are called weights.


Summing up the areas under the quadratic approximants across subintervals yields Simpson's
rule:

$$\int_a^b f(x)dx \approx \sum_{i=0}^{n-1} w_if(x_i)\thinspace .$$

where $w_1 = w_n = h/3$ and, otherwise, $w_i = 4h/3$ if i is even and $w_i = 2h/3$ if i is odd.

More formally, let $x_i = a + (i - 1)h$ for $i = 1; 2; ... ; n$, where
$ h=(b − a) / (n-1)$ and $n$ is odd. The nodes $x_i$ divide the interval $[a; b]$ into an even number
$n - 1$ of subintervals of equal length $h$.

Simpson's rule is as simple as the trapezoid rule, and thus not much harder to program.
Even though Simpson's rule is based on *locally quadratic approximation* of the integrand,
it is *third order exact*. That is, it exactly computes the integral of any cubic polynomial.
In general, if the integrand is smooth, Simpson's rule yields an approximation error that is
$O(h^4)$, and thus falls at twice the geometric rate of the error associated with the trapezoid
rule.


*Simpson's rule is preferred to the trapezoid rule* when the integrand $f$ is smooth because
it retains the algorithmic simplicity of the trapezoid rule while offering **twice the degree of
approximation**. However, the trapezoid rule will often be more accurate than Simpson's rule
if the integrand exhibits **discontinuities in its first derivative**, which can occur in economic
applications exhibiting corner solutions. Newton-Cotes rules based on fourth and higher
order piecewise polynomial approximations exist, but are more diÆcult to work with and
thus are rarely used.

### Univariate Simpson's rule

In [5]:
def _qnwsimp1(n, a, b):
    """
    Compute univariate Simpson quadrature nodes and weights
    Parameters
    ----------
    n : int
        The number of nodes
    a : int
        The lower endpoint
    b : int
        The upper endpoint
    Returns
    -------
    nodes : np.ndarray(dtype=float)
        An n element array of nodes
    nodes : np.ndarray(dtype=float)
        An n element array of weights
    Notes
    -----
    Based of original function ``qnwsimp1`` in CompEcon toolbox by
    Miranda and Fackler
    References
    ----------
    Miranda, Mario J, and Paul L Fackler. Applied Computational
    Economics and Finance, MIT Press, 2002.
    """
    if n % 2 == 0:
        print("WARNING qnwsimp: n must be an odd integer. Increasing by 1")
        n += 1

    nodes = np.linspace(a, b, n)
    dx = nodes[1] - nodes[0]
    weights = np.tile([2.0, 4.0], (n + 1) // 2)
    weights = weights[:n]
    weights[0] = weights[-1] = 1
    weights = (dx / 3.0) * weights

    return nodes, weights    

Through the use of tensor product principles, univariate Newton-Cotes quadrature schemes
can be generalized for higher dimensional integration.


This construction
principle can be applied to an arbitrary dimension using repeated tensor product operations.

In [ ]:
def qnwsimp(n, a, b):
    """
    Computes multivariate Simpson quadrature nodes and weights.
    Parameters
    ----------
    n : int or array_like(float)
        A length-d iterable of the number of nodes in each dimension
    a : scalar or array_like(float)
        A length-d iterable of lower endpoints. If a scalar is given,
        that constant is repeated d times, where d is the number of
        dimensions
    b : scalar or array_like(float)
        A length-d iterable of upper endpoints. If a scalar is given,
        that constant is repeated d times, where d is the number of
        dimensions
    Returns
    -------
    nodes : np.ndarray(dtype=float)
        Quadrature nodes
    weights : np.ndarray(dtype=float)
        Weights for quadrature nodes
    Notes
    -----
    Based of original function ``qnwsimp`` in CompEcon toolbox by
    Miranda and Fackler
    References
    ----------
    Miranda, Mario J, and Paul L Fackler. Applied Computational
    Economics and Finance, MIT Press, 2002.
    """
    return _make_multidim_func(_qnwsimp1, n, a, b)

In most computational economic applications it is not possible to determine a priori
how many partition points are needed to compute an integral to a desired level of accuracy
using a Newton-Cotes quadrature rule. One solution to this problem is to use an **adaptive
quadrature strategy** whereby one increases the number of points at which the integrand is
evaluated until the sequence of estimates of the integral converge. EÆcient adaptive Newton-
Cotes quadrature schemes are especially easy to implement. One simple, but powerful,
scheme calls for the number of intervals to be doubled with each iteration. Because the new
partition points include the partition points used in the previous iteration, the computational
effort required to form the new integral estimate is cut in half. More sophisticated adaptive
Newton-Cotes quadrature techniques relax the requirement that the intervals be equally
spaced and concentrate new evaluation points in those areas where the integrand appears to
be most irregular.

## 5.2 Gaussian Quadrature
Gaussian quadrature rules are constructed with respect to specific weighting functions.
Specifically, for a weighting function $w$ defined on an interval $I \in R$ of the real
line, and for a given order of approximation n, the quadrature nodes $x_1; x_2; ... ; x_n$
and quadrature weights $w_1; w_2; ...; w_n$ are chosen so as to satisfy the $2n$ "moment-matching"
conditions:


reference 

http://compeconworkshop.org/PDF/Part04.pdf

https://people.ucsc.edu/~ealdrich/Teaching/ComputationGroup/Slides/lec3.pdf

http://www.econ.nyu.edu/user/violante/NYUTeaching/QM/Spring15/Lectures/Lecture6_Integration_Slides.pdf


![](https://upload.wikimedia.org/wikipedia/commons/thumb/9/93/Comparison_Gaussquad_trapezoidal.svg/880px-Comparison_Gaussquad_trapezoidal.svg.png)



*Comparison between 2-point Gaussian and trapezoidal quadrature. The blue line is the polynomial ${\displaystyle y(x)=7x^{3}-8x^{2}-3x+3}$ , whose integral in [−1, 1] is 2/3. The trapezoidal rule returns the integral of the orange dashed line, equal to ${\displaystyle y(-1)+y(1)=-10} $. The 2-point Gaussian quadrature rule returns the integral of the black dashed curve, equal to ${\displaystyle y(-{\sqrt {\scriptstyle 1/3}})+y({\sqrt {\scriptstyle 1/3}})=2/3}$. Such a result is exact, since the green region has the same area as the red regions.*

reference

https://en.wikipedia.org/wiki/Gaussian_quadrature

http://mathworld.wolfram.com/GaussianQuadrature.html

In numerical analysis, a quadrature rule is an approximation of the definite integral of a function, usually stated as a weighted sum of function values at specified points within the domain of integration. (See numerical integration for more on quadrature rules.) An n-point Gaussian quadrature rule, named after Carl Friedrich Gauss, is a quadrature rule constructed to yield an exact result for polynomials of degree $2n − 1$ or less by a suitable choice of the points xi and weights wi for $i = 1, ..., n$. The domain of integration for such a rule is conventionally taken as $[−1, 1]$, so the rule is stated as
$${\displaystyle \int _{-1}^{1}f(x)\,dx=\sum _{i=1}^{n}w_{i}f(x_{i}).} $$.


if the integrated function can be written as ${\displaystyle f(x)=\omega (x)g(x)}$, where $g(x)$ is approximately polynomial and $ω(x)$ is known, then alternative weights ${\displaystyle w_{i}'} $ and points ${\displaystyle x_{i}'} $ that depend on the weighting function $ω(x)$ may give better results, where
$${\displaystyle \int _{-1}^{1}f(x)\,dx=\int _{-1}^{1}\omega (x)g(x)\,dx\approx \sum _{i=1}^{n}w_{i}'g(x_{i}').} $$.



Common weighting functions include ${\displaystyle \omega (x)=1/{\sqrt {1-x^{2}}}} $ (Chebyshev–Gauss) and ${\displaystyle \omega (x)=e^{-x^{2}}} $ (Gauss–Hermite).




In [ ]:
#https://github.com/birocoles/Disciplina-metodos-computacionais/blob/master/Content/newton-cotes.ipynb




# http://nbviewer.jupyter.org/github/sbustamante/ComputationalMethods/blob/master/material/numerical-calculus.ipynb#Numerical-Integration

## 5.3 Monte Carlo Integration

In numerical integration, methods such as the trapezoidal rule use a deterministic approach. Monte Carlo integration, on the other hand, employs a non-deterministic approach: each realization provides a different outcome.


![](https://upload.wikimedia.org/wikipedia/commons/thumb/2/20/MonteCarloIntegrationCircle.svg/440px-MonteCarloIntegrationCircle.svg.png)




Monte Carlo integration methods are motivated by the Strong Law of Large Numbers.
One version of the Law states that if $x_1; x_2; ... ; x_n$ are independent realizations of a
random variable $\tilde X$
and $f$ is a continuous function, then

$$\lim _{{n\to \infty }} \frac{1}{n} \sum_{i=1}^{n}  f(x_i) =  E f(\tilde X)$$

with probability one.

The Monte Carlo integration scheme is thus a simple one. To compute an approximation to the expectation of $f(\tilde X)$, one draws a random sample $x_1; x_2; ... ; x_n$ from the distribution of $\tilde X$ and sets



$$  E f(\tilde X) \approx \frac{1}{n} \sum_{i=1}^{n}  f(x_i) $$

Most numerical software packages provide a routine that generates pseudo-random variables
that are uniformly distributed on the interval [0; 1]. A uniform random number generator
is useful for generating random samples from other distributions. Suppose $\tilde X$ has a cumulative distribution function  

$$ F(x) = Pr( \tilde X \le x) $$


whose inverse has a well-defined closed form. If $\tilde U$ is uniformly distributed on $(0; 1)$, then
$F^{-1}( \tilde U)$ has the same distribution as $\tilde X$.

Most numerical software packages also provide an intrinsic routine that generates pseudorandom
standard normal variables. The routine may also be used to generate pseudo-random sequences of lognormal and multivariate normal variables.



In [ ]:
def qnwlogn(n, mu=None, sig2=None):
    """
    Computes nodes and weights for multivariate lognormal distribution
    Parameters
    ----------
    n : int or array_like(float)
        A length-d iterable of the number of nodes in each dimension
    mu : scalar or array_like(float), optional(default=zeros(d))
        The means of each dimension of the random variable. If a scalar
        is given, that constant is repeated d times, where d is the
        number of dimensions
    sig2 : array_like(float), optional(default=eye(d))
        A d x d array representing the variance-covariance matrix of the
        multivariate normal distribution.
    Returns
    -------
    nodes : np.ndarray(dtype=float)
        Quadrature nodes
    weights : np.ndarray(dtype=float)
        Weights for quadrature nodes
    Notes
    -----
    Based of original function ``qnwlogn`` in CompEcon toolbox by
    Miranda and Fackler
    References
    ----------
    Miranda, Mario J, and Paul L Fackler. Applied Computational
    Economics and Finance, MIT Press, 2002.
    """
    nodes, weights = qnwnorm(n, mu, sig2)
    return np.exp(nodes), weights

A fundamental problem that arises with Monte Carlo integration is that it is almost impossible
to generate a truly random sample of variates for any distribution. Most compilers
and vector processing packages provide intrinsic routines for computing so-called random
numbers. These routines, however, employ iteration rules that generate a purely deterministic,
not random, sequence of numbers. In particular, if the generator is repeatedly initiated
at the same point, it will return the same sequence of "random" variates each time. About
all that can be said of numerical random number generators is that good ones will generate
sequences that appear to be random, in that they pass certain statistical tests for randomness.
For this reason, numerical random number generators are more accurately said to
generate sequences of *"pseudo-random"* rather than random numbers.

Monte Carlo integration is easy to implement andmay be preferred over Gaussian quadrature
if the a routine for computing the Gaussian mass points and probabilities is not readily
available or if the integration is over many dimensions. Monte Carlo integration, however, is
subject to a sampling error that cannot be bounded with certainty. The approximation can
be made more accurate, in a dubious statistical sense, by increasing the size of the random
sample, but this can be expensive if evaluating f or generating the pseudo-random variate is
costly. Approximations generated by Monte Carlo integration will vary from one integration
to the next, unless initiated at the same point, making the use of Monte Carlo integration
in conjunction within other iterative schemes, such as dynamic programming or maximum
likelihood estimation, problematic. So-called quasi Monte-Carlo methods can circumvent
some of the problems associated with Monte-Carlo integration.

Quasi-Monte Carlo uses a low-discrepancy sequence such as the Halton sequence, the Sobol sequence, or the Faure sequence, whereas Monte Carlo uses a pseudorandom sequence. The advantage of using low-discrepancy sequences is a faster rate of convergence. Quasi-Monte Carlo has a rate of convergence close to $O(1/N)$, whereas the rate for the Monte Carlo method is $O(N^{−0.5})$.






The Quasi-Monte Carlo method recently became popular in the area of mathematical finance or computational finance. In these areas, high-dimensional numerical integrals, where the integral should be evaluated within a threshold ε, occur frequently. Hence, the Monte Carlo method and the quasi-Monte Carlo method are beneficial in these situations.

https://en.wikipedia.org/wiki/Quasi-Monte_Carlo_method

## 5.4 Quasi-Monte Carlo Integration


Although Monte-Carlo integration methods originated using insights from probability
theory, recent extensions have severed that connection and, in the process, demonstrated
ways in which the methods can be improved. Monte-Carlo methods rely on
sequences $x_i$ with the property that


$$\lim _{{n\to \infty }} \frac{b-a}{n} \sum_{i=1}^{n}  f(x_i) =  \int_{a}^{b} f(x) dx$$


without regard to whether the sequences passes standard tests of randomness. Any sequence
that satisfies this condition for arbitrary (Riemann) integrable functions can be used to approximate an integral on [a; b]. Although the Law of Large Numbers assures us that this is true when the $x_i$ are independent and identically distributed random variables, other
sequences also satisfy this property. Indeed, it can be shown that sequences that are explicitly
non-random, but instead attempt to fill in space in a regular manner can often provide more
accurate approximations to definite integrals.


In [ ]:
def _qnwnorm1(n):
    """
    Compute nodes and weights for quadrature of univariate standard
    normal distribution
    Parameters
    ----------
    n : int
        The number of nodes
    Returns
    -------
    nodes : np.ndarray(dtype=float)
        An n element array of nodes
    nodes : np.ndarray(dtype=float)
        An n element array of weights
    Notes
    -----
    Based of original function ``qnwnorm1`` in CompEcon toolbox by
    Miranda and Fackler
    References
    ----------
    Miranda, Mario J, and Paul L Fackler. Applied Computational
    Economics and Finance, MIT Press, 2002.
    """
    maxit = 100
    pim4 = 1 / np.pi**(0.25)
    m = np.fix((n + 1) / 2).astype(int)
    nodes = np.zeros(n)
    weights = np.zeros(n)

    for i in range(m):
        if i == 0:
            z = np.sqrt(2*n+1) - 1.85575 * ((2 * n + 1)**(-1 / 6.1))
        elif i == 1:
            z = z - 1.14 * (n ** 0.426) / z
        elif i == 2:
            z = 1.86 * z + 0.86 * nodes[0]
        elif i == 3:
            z = 1.91 * z + 0.91 * nodes[1]
        else:
            z = 2 * z + nodes[i-2]

        its = 0

        while its < maxit:
            its += 1
            p1 = pim4
            p2 = 0
            for j in range(1, n+1):
                p3 = p2
                p2 = p1
                p1 = z * math.sqrt(2.0/j) * p2 - math.sqrt((j - 1.0) / j) * p3

            pp = math.sqrt(2 * n) * p2
            z1 = z
            z = z1 - p1/pp
            if abs(z - z1) < 1e-14:
                break

        if its == maxit:
            raise ValueError("Failed to converge in _qnwnorm1")

        nodes[n - 1 - i] = z
        nodes[i] = -z
        weights[i] = 2 / (pp*pp)
        weights[n - 1 - i] = weights[i]

    weights /= math.sqrt(math.pi)
    nodes = nodes * math.sqrt(2.0)

    return nodes, weights

def _qnwbeta1(n, a=1.0, b=1.0):
    """
    Computes nodes and weights for quadrature on the beta distribution.
    Default is a=b=1 which is just a uniform distribution
    NOTE: For now I am just following compecon; would be much better to
    find a different way since I don't know what they are doing.
    Parameters
    ----------
    n : scalar : int
        The number of quadrature points
    a : scalar : float, optional(default=1)
        First Beta distribution parameter
    b : scalar : float, optional(default=1)
        Second Beta distribution parameter
    Returns
    -------
    nodes : np.ndarray(dtype=float, ndim=1)
        The quadrature points
    weights : np.ndarray(dtype=float, ndim=1)
        The quadrature weights that correspond to nodes
    Notes
    -----
    Based of original function ``_qnwbeta1`` in CompEcon toolbox by
    Miranda and Fackler
    References
    ----------
    Miranda, Mario J, and Paul L Fackler. Applied Computational
    Economics and Finance, MIT Press, 2002.
    """
    # We subtract one and write a + 1 where we actually want a, and a
    # where we want a - 1
    a = a - 1
    b = b - 1

    maxiter = 25

    # Allocate empty space
    nodes = np.zeros(n)
    weights = np.zeros(n)

    # Find "reasonable" starting values.  Why these numbers?
    for i in range(n):
        if i == 0:
            an = a/n
            bn = b/n
            r1 = (1+a) * (2.78/(4+n*n) + .768*an/n)
            r2 = 1 + 1.48*an + .96*bn + .452*an*an + .83*an*bn
            z = 1 - r1/r2
        elif i == 1:
            r1 = (4.1+a) / ((1+a)*(1+0.156*a))
            r2 = 1 + 0.06 * (n-8) * (1+0.12*a)/n
            r3 = 1 + 0.012*b * (1+0.25*abs(a))/n
            z = z - (1-z) * r1 * r2 * r3
        elif i == 2:
            r1 = (1.67+0.28*a)/(1+0.37*a)
            r2 = 1+0.22*(n-8)/n
            r3 = 1+8*b/((6.28+b)*n*n)
            z = z-(nodes[0]-z)*r1*r2*r3
        elif i == n - 2:
            r1 = (1+0.235*b)/(0.766+0.119*b)
            r2 = 1/(1+0.639*(n-4)/(1+0.71*(n-4)))
            r3 = 1/(1+20*a/((7.5+a)*n*n))
            z = z+(z-nodes[-4])*r1*r2*r3
        elif i == n - 1:
            r1 = (1+0.37*b) / (1.67+0.28*b)
            r2 = 1 / (1+0.22*(n-8)/n)
            r3 = 1 / (1+8*a/((6.28+a)*n*n))
            z = z+(z-nodes[-3])*r1*r2*r3
        else:
            z = 3*nodes[i-1] - 3*nodes[i-2] + nodes[i-3]

        ab = a+b

        # Root finding
        its = 0
        z1 = -100
        while abs(z - z1) > 1e-10 and its < maxiter:
            temp = 2 + ab
            p1 = (a-b + temp*z)/2
            p2 = 1

            for j in range(2, n+1):
                p3 = p2
                p2 = p1
                temp = 2*j + ab
                aa = 2*j * (j+ab)*(temp-2)
                bb = (temp-1) * (a*a - b*b + temp*(temp-2) * z)
                c = 2 * (j - 1 + a) * (j - 1 + b) * temp
                p1 = (bb*p2 - c*p3)/aa

            pp = (n*(a-b-temp*z) * p1 + 2*(n+a)*(n+b)*p2)/(temp*(1 - z*z))
            z1 = z
            z = z1 - p1/pp

            if abs(z - z1) < 1e-12:
                break

            its += 1

        if its == maxiter:
            raise ValueError("Max Iteration reached.  Failed to converge")

        nodes[i] = z
        weights[i] = temp/(pp*p2)

    nodes = (1-nodes)/2
    weights = weights * math.exp(gammaln(a+n) + gammaln(b+n)
                                 - gammaln(n+1) - gammaln(n+ab+1))
    weights = weights / (2*math.exp(gammaln(a+1) + gammaln(b+1)
                         - gammaln(ab+2)))

    return nodes, weights


def _qnwgamma1(n, a=None):
    """
    Insert docs.  Default is a=0
    NOTE: For now I am just following compecon; would be much better to
    find a different way since I don't know what they are doing.
    Parameters
    ----------
    n : scalar : int
        The number of quadrature points
    a : scalar : float
        Gamma distribution parameter
    Returns
    -------
    nodes : np.ndarray(dtype=float, ndim=1)
        The quadrature points
    weights : np.ndarray(dtype=float, ndim=1)
        The quadrature weights that correspond to nodes
    Notes
    -----
    Based of original function ``qnwgamma1`` in CompEcon toolbox by
    Miranda and Fackler
    References
    ----------
    Miranda, Mario J, and Paul L Fackler. Applied Computational
    Economics and Finance, MIT Press, 2002.
    """
    if a is None:
        a = 0
    else:
        a -= 1

    maxit = 10

    factor = -math.exp(gammaln(a+n) - gammaln(n) - gammaln(a+1))
    nodes = np.zeros(n)
    weights = np.zeros(n)

    # Create nodes
    for i in range(n):
        # Reasonable starting values
        if i == 0:
            z = (1+a) * (3+0.92*a) / (1 + 2.4*n + 1.8*a)
        elif i == 1:
            z = z + (15 + 6.25*a) / (1 + 0.9*a + 2.5*n)
        else:
            j = i-1
            z = z + ((1 + 2.55*j) / (1.9*j) + 1.26*j*a / (1 + 3.5*j)) * \
                (z - nodes[j-1]) / (1 + 0.3*a)

        # root finding iterations
        its = 0
        z1 = -10000
        while abs(z - z1) > 1e-10 and its < maxit:
            p1 = 1.0
            p2 = 0.0
            for j in range(1, n+1):
                p3 = p2
                p2 = p1
                p1 = ((2*j - 1 + a - z)*p2 - (j - 1 + a)*p3) / j

            pp = (n*p1 - (n+a)*p2) / z
            z1 = z
            z = z1 - p1/pp
            its += 1

        if its == maxit:
            raise ValueError('Failure to converge')

        nodes[i] = z
        weights[i] = factor / (pp*n*p2)

    return nodes, weights

## 5.5 An Integration Toolbox
The Matlab toolbox accompanying the textbook includes four functions for computing
numerical integrals for general functions. Each takes three inputs, n, a, and b and
generates appropriate nodes and weights. The functions `qnwtrap` and `qnwsimp` implement the Newton-Cotes trapezoid and Simpson's rule methods, `qnwlege` implements
Gauss-Legendre quadrature and `qnwequi` generates nodes and weights associated with
either equidistributed or pseudo-random sequences. The calling syntax is the same
for each and is illustrated with below with `qnwtrap`.




In [ ]:
     







def qnwlege(n, a, b):
    """
    Computes multivariate Guass-Legendre  quadrature nodes and weights.
    Parameters
    ----------
    n : int or array_like(float)
        A length-d iterable of the number of nodes in each dimension
    a : scalar or array_like(float)
        A length-d iterable of lower endpoints. If a scalar is given,
        that constant is repeated d times, where d is the number of
        dimensions
    b : scalar or array_like(float)
        A length-d iterable of upper endpoints. If a scalar is given,
        that constant is repeated d times, where d is the number of
        dimensions
    Returns
    -------
    nodes : np.ndarray(dtype=float)
        Quadrature nodes
    weights : np.ndarray(dtype=float)
        Weights for quadrature nodes
    Notes
    -----
    Based of original function ``qnwlege`` in CompEcon toolbox by
    Miranda and Fackler
    References
    ----------
    Miranda, Mario J, and Paul L Fackler. Applied Computational
    Economics and Finance, MIT Press, 2002.
    """
    return _make_multidim_func(_qnwlege1, n, a, b)



def qnwequi(n, a, b, kind="N", equidist_pp=None):
    """
    Generates equidistributed sequences with property that averages
    value of integrable function evaluated over the sequence converges
    to the integral as n goes to infinity.
    Parameters
    ----------
    n : int
        Number of sequence points
    a : scalar or array_like(float)
        A length-d iterable of lower endpoints. If a scalar is given,
        that constant is repeated d times, where d is the number of
        dimensions
    b : scalar or array_like(float)
        A length-d iterable of upper endpoints. If a scalar is given,
        that constant is repeated d times, where d is the number of
        dimensions
    kind : string, optional(default="N")
        One of the following:
        - N - Neiderreiter (default)
        - W - Weyl
        - H - Haber
        - R - pseudo Random
    equidist_pp : array_like, optional(default=None)
        TODO: I don't know what this does
    Returns
    -------
    nodes : np.ndarray(dtype=float)
        Quadrature nodes
    weights : np.ndarray(dtype=float)
        Weights for quadrature nodes
    Notes
    -----
    Based of original function ``qnwequi`` in CompEcon toolbox by
    Miranda and Fackler
    References
    ----------
    Miranda, Mario J, and Paul L Fackler. Applied Computational
    Economics and Finance, MIT Press, 2002.
    """
    if equidist_pp is None:
        equidist_pp = np.sqrt(np.array(list(sym.primerange(0, 7920))))

    n, a, b = list(map(np.atleast_1d, list(map(np.asarray, [n, a, b]))))

    d = max(list(map(len, [n, a, b])))
    n = np.prod(n)

    if a.size == 1:
        a = np.repeat(a, d)

    if b.size == 1:
        b = np.repeat(b, d)

    i = np.arange(1, n + 1)

    if kind.upper() == "N":  # Neiderreiter
        j = 2.0 ** (np.arange(1, d+1) / (d+1))
        nodes = np.outer(i, j)
        nodes = (nodes - np.fix(nodes)).squeeze()
    elif kind.upper() == "W":  # Weyl
        j = equidist_pp[:d]
        nodes = np.outer(i, j)
        nodes = (nodes - np.fix(nodes)).squeeze()
    elif kind.upper() == "H":  # Haber
        j = equidist_pp[:d]
        nodes = np.outer(i * (i+1) / 2, j)
        nodes = (nodes - np.fix(nodes)).squeeze()
    elif kind.upper() == "R":  # pseudo-random
        nodes = np.random.rand(n, d).squeeze()
    else:
        raise ValueError("Unknown sequence requested")

    # compute nodes and weights
    r = b - a
    nodes = a + nodes * r
    weights = (np.prod(r) / n) * np.ones(n)

    return nodes, weights

def qnwnorm(n, mu=None, sig2=None, usesqrtm=False):
    """
    Computes nodes and weights for multivariate normal distribution
    Parameters
    ----------
    n : int or array_like(float)
        A length-d iterable of the number of nodes in each dimension
    mu : scalar or array_like(float), optional(default=zeros(d))
        The means of each dimension of the random variable. If a scalar
        is given, that constant is repeated d times, where d is the
        number of dimensions
    sig2 : array_like(float), optional(default=eye(d))
        A d x d array representing the variance-covariance matrix of the
        multivariate normal distribution.
    Returns
    -------
    nodes : np.ndarray(dtype=float)
        Quadrature nodes
    weights : np.ndarray(dtype=float)
        Weights for quadrature nodes
    Notes
    -----
    Based of original function ``qnwnorm`` in CompEcon toolbox by
    Miranda and Fackler
    References
    ----------
    Miranda, Mario J, and Paul L Fackler. Applied Computational
    Economics and Finance, MIT Press, 2002.
    """
    n = np.asarray(n)
    d = n.size

    if mu is None:
        mu = np.zeros(d)
    else:
        mu = np.asarray(mu)

    if sig2 is None:
        sig2 = np.eye(d)
    else:
        sig2 = np.asarray(sig2).reshape(d, d)

    if all([x.size == 1 for x in [n, mu, sig2]]):
        nodes, weights = _qnwnorm1(n)
    else:
        nodes = []
        weights = []

        for i in range(d):
            _1d = _qnwnorm1(n[i])
            nodes.append(_1d[0])
            weights.append(_1d[1])

        nodes = gridmake(*nodes)
        weights = ckron(*weights[::-1])

    if usesqrtm:
        new_sig2 = la.sqrtm(sig2)
    else:  # cholesky
        new_sig2 = la.cholesky(sig2)

    if d > 1:
        nodes = nodes.dot(new_sig2) + mu  # Broadcast ok
    else:  # nodes.dot(sig) will not be aligned in scalar case.
        nodes = nodes * new_sig2 + mu

    return nodes.squeeze(), weights


def qnwlogn(n, mu=None, sig2=None):
    """
    Computes nodes and weights for multivariate lognormal distribution
    Parameters
    ----------
    n : int or array_like(float)
        A length-d iterable of the number of nodes in each dimension
    mu : scalar or array_like(float), optional(default=zeros(d))
        The means of each dimension of the random variable. If a scalar
        is given, that constant is repeated d times, where d is the
        number of dimensions
    sig2 : array_like(float), optional(default=eye(d))
        A d x d array representing the variance-covariance matrix of the
        multivariate normal distribution.
    Returns
    -------
    nodes : np.ndarray(dtype=float)
        Quadrature nodes
    weights : np.ndarray(dtype=float)
        Weights for quadrature nodes
    Notes
    -----
    Based of original function ``qnwlogn`` in CompEcon toolbox by
    Miranda and Fackler
    References
    ----------
    Miranda, Mario J, and Paul L Fackler. Applied Computational
    Economics and Finance, MIT Press, 2002.
    """
    nodes, weights = qnwnorm(n, mu, sig2)
    return np.exp(nodes), weights




def qnwunif(n, a, b):
    """
    Computes quadrature nodes and weights for multivariate uniform
    distribution
    Parameters
    ----------
    n : int or array_like(float)
        A length-d iterable of the number of nodes in each dimension
    a : scalar or array_like(float)
        A length-d iterable of lower endpoints. If a scalar is given,
        that constant is repeated d times, where d is the number of
        dimensions
    b : scalar or array_like(float)
        A length-d iterable of upper endpoints. If a scalar is given,
        that constant is repeated d times, where d is the number of
        dimensions
    Returns
    -------
    nodes : np.ndarray(dtype=float)
        Quadrature nodes
    weights : np.ndarray(dtype=float)
        Weights for quadrature nodes
    Notes
    -----
    Based of original function ``qnwunif`` in CompEcon toolbox by
    Miranda and Fackler
    References
    ----------
    Miranda, Mario J, and Paul L Fackler. Applied Computational
    Economics and Finance, MIT Press, 2002.
    """
    n, a, b = list(map(np.asarray, [n, a, b]))
    nodes, weights = qnwlege(n, a, b)
    weights = weights / np.prod(b - a)
    return nodes, weights




def quadrect(f, n, a, b, kind='lege', *args, **kwargs):
    """
    Integrate the d-dimensional function f on a rectangle with lower and
    upper bound for dimension i defined by a[i] and b[i], respectively;
    using n[i] points.
    Parameters
    ----------
    f : function
        The function to integrate over. This should be a function
        that accepts as its first argument a matrix representing points
        along each dimension (each dimension is a column). Other
        arguments that need to be passed to the function are caught by
        `*args` and `**kwargs`
    n : int or array_like(float)
        A length-d iterable of the number of nodes in each dimension
    a : scalar or array_like(float)
        A length-d iterable of lower endpoints. If a scalar is given,
        that constant is repeated d times, where d is the number of
        dimensions
    b : scalar or array_like(float)
        A length-d iterable of upper endpoints. If a scalar is given,
        that constant is repeated d times, where d is the number of
        dimensions
    kind : string, optional(default='lege')
        Specifies which type of integration to perform. Valid
        values are:
        lege - Gauss-Legendre
        cheb - Gauss-Chebyshev
        trap - trapezoid rule
        simp - Simpson rule
        N    - Neiderreiter equidistributed sequence
        W    - Weyl equidistributed sequence
        H    - Haber  equidistributed sequence
        R    - Monte Carlo
    *args, **kwargs :
        Other arguments passed to the function f
    Returns
    -------
    out : scalar (float)
        The value of the integral on the region [a, b]
    Notes
    -----
    Based of original function ``quadrect`` in CompEcon toolbox by
    Miranda and Fackler
    References
    ----------
    Miranda, Mario J, and Paul L Fackler. Applied Computational
    Economics and Finance, MIT Press, 2002.
    """
    if kind.lower() == "lege":
        nodes, weights = qnwlege(n, a, b)
    elif kind.lower() == "cheb":
        nodes, weights = qnwcheb(n, a, b)
    elif kind.lower() == "trap":
        nodes, weights = qnwtrap(n, a, b)
    elif kind.lower() == "simp":
        nodes, weights = qnwsimp(n, a, b)
    else:
        nodes, weights = qnwequi(n, a, b, kind)

    out = weights.dot(f(nodes, *args, **kwargs))
    return out


def qnwbeta(n, a=1.0, b=1.0):
    """
    Computes nodes and weights for beta distribution
    Parameters
    ----------
    n : int or array_like(float)
        A length-d iterable of the number of nodes in each dimension
    a : scalar or array_like(float), optional(default=1.0)
        A length-d
    b : array_like(float), optional(default=1.0)
        A d x d array representing the variance-covariance matrix of the
        multivariate normal distribution.
    Returns
    -------
    nodes : np.ndarray(dtype=float)
        Quadrature nodes
    weights : np.ndarray(dtype=float)
        Weights for quadrature nodes
    Notes
    -----
    Based of original function ``qnwbeta`` in CompEcon toolbox by
    Miranda and Fackler
    References
    ----------
    Miranda, Mario J, and Paul L Fackler. Applied Computational
    Economics and Finance, MIT Press, 2002.
    """
    return _make_multidim_func(_qnwbeta1, n, a, b)


def qnwgamma(n, a=None):
    """
    Computes nodes and weights for gamma distribution
    Parameters
    ----------
    n : int or array_like(float)
        A length-d iterable of the number of nodes in each dimension
    mu : scalar or array_like(float), optional(default=zeros(d))
        The means of each dimension of the random variable. If a scalar
        is given, that constant is repeated d times, where d is the
        number of dimensions
    sig2 : array_like(float), optional(default=eye(d))
        A d x d array representing the variance-covariance matrix of the
        multivariate normal distribution.
    Returns
    -------
    nodes : np.ndarray(dtype=float)
        Quadrature nodes
    weights : np.ndarray(dtype=float)
        Weights for quadrature nodes
    Notes
    -----
    Based of original function ``qnwgamma`` in CompEcon toolbox by
    Miranda and Fackler
    References
    ----------
    Miranda, Mario J, and Paul L Fackler. Applied Computational
    Economics and Finance, MIT Press, 2002.
    """
    return _make_multidim_func(_qnwgamma1, n, a)






All of the quadrature functions will use tensor products to generate nodes and
weights for integration over an arbitrary bounded interval [a; b] in higher dimensional
spaces.

## 5.6 Numerical Differentiation

![](https://upload.wikimedia.org/wikipedia/commons/thumb/1/18/Derivative.svg/460px-Derivative.svg.png)


The most natural way to approximate a derivative is to replace it with a finite difference.
The definition of a derivative,

$$ f'(x)=\lim _{{h\to 0}}{f(x+h)-f(x) \over h}. $$




Another two-point formula is to compute the slope of a nearby secant line through the points $(x-h,f(x-h))$ and $(x+h,f(x+h))$. The slope of this line is
$${\displaystyle {f(x+h)-f(x-h) \over 2h}.} $$
This formula is known as the symmetric difference quotient. In this case the first-order errors cancel, so the slope of these secant lines differ from the slope of the tangent line by an amount that is approximately proportional to ${\displaystyle h^{2}}$. Hence for small values of h this is a more accurate approximation to the tangent line than the one-sided estimation. 

## 5.7 Initial Value Problems
Differential equations pose the problem of inferring a function given information
about its derivatives and additional "boundary" conditions. Differential equations
may characterized as either ordinary differential equations (ODEs), whose solutions
are functions of a single argument, and partial differential equations (PDEs), whose
solutions are functions of multiple arguments. Both ODEs and PDEs may be solved
numerically using finite difference methods.

From a numerical point of view the distinction between ODEs and PDEs is less
important than the distinction between initial value problems (IVPs), which can be
solved in a recursive or evolutionary fashion, and boundary value problems (BVPs),
which require the entire solution to be computed simultaneously because the solution
at one point (in time and/or space) depends on the solution everywhere else. For
ODEs, the solution of an IVP is known at some point and the solution near this
point can then be (approximately) determined. This, in turn, allows the solution at
still other points to be approximated and so forth. BVPs, on the other hand, require
simultaneous solution of the differential equation and the boundary conditions. We
take up the solution of IVPs in this section, but defer discussion of BVPs until the
next chapter (page 164).

There are numerous other approaches and refinements to solving initial value problems.
Brie
y, these include so-called multi-step algorithms which utilize information
from previous steps to determine the current step direction (Runge-Kutta are singlestep
methods). Also, any method can adapt the step size to the current behavior
of the system by monitoring the truncation error, reducing (increasing) the step size
if this error is unacceptably large (small). Adaptive schemes are important if one
requires a given level of accuracy.

### Example: Commercial Fishery